# Lab 4: Urban Heat Island Detection using Thermal Satellite Imagery

## Goal
In this exercise, you will:
1. Select and compare two thermal satellite images from Kraków (or another city that has recently undergone intensive, thoughtless concrete development) – one from **2013** and one from **2024**.
2. Each image must be acquired on a **hot summer day** (T > 27°C based on IMGW meteorological data).
3. Images must have **low cloud cover** (< 20%).
4. Visualize thermal data and analyze surface temperature differences (Urban Heat Island effect).

## Task 1: Area of Interest (AOI)
- Define an AOI over the Reduta Street area in Kraków.
- Use `ee.Geometry.Polygon` or `ee.Geometry.Rectangle`.

In [39]:
#import ee
#ee.Authenticate()
ee.Initialize(project='ee-kkosciukk')

In [60]:
aoi = ee.Geometry.Rectangle([19.947306, 50.097058, 19.969359, 50.103414])


In [43]:
print(aoi.getInfo())

{'type': 'Polygon', 'coordinates': [[[19.947306, 50.097058], [19.969359, 50.097058], [19.969359, 50.103414], [19.947306, 50.103414], [19.947306, 50.097058]]]}


## Task 2: Download and Analyze IMGW Meteorological Data
- Visit: [IMGW Archive](https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/)
- Navigate to:
  - `dane_meteorologiczne/dobowe/synop/2013/`
  - `dane_meteorologiczne/dobowe/synop/2024/`
- Download `s_d_tmax.csv` for both years.
- Filter the rows for **station ID 12566 (e.g. Kraków-Balice)**.
- Identify days in **July or August** with **TMAX > 27°C**.

In [ ]:
import pandas as pd

df = pd.read_csv("k_d_08_2013.csv", header=None, encoding="latin1")


df.columns = [
    "station_id", "station_name", "year", "month", "day", "tmax",
    "col6", "col7", "col8", "col9", "col10", "col11",
    "col12", "col13", "col14", "col15", "col16", "col17"
]


krakow_obs = df[df["station_name"] == "KRAKÓW-OBSERWATORIUM"]


hot_days = krakow_obs[(krakow_obs["month"] == 8) & (krakow_obs["tmax"] > 27)]


print(hot_days[["year", "month", "day", "tmax"]])



      year  month  day  tmax
2170  2013      8    1  27.2
2171  2013      8    2  31.0
2172  2013      8    3  32.8
2173  2013      8    4  30.7
2174  2013      8    5  31.5
2175  2013      8    6  34.2
2176  2013      8    7  37.0
2177  2013      8    8  38.3
2178  2013      8    9  31.4
2186  2013      8   17  29.0
2187  2013      8   18  31.8
2188  2013      8   19  32.5


In [ ]:
import pandas as pd


df = pd.read_csv("k_d_08_2024.csv", header=None, encoding="latin1")


df.columns = [
    "station_id", "station_name", "year", "month", "day", "tmax",
    "col6", "col7", "col8", "col9", "col10", "col11",
    "col12", "col13", "col14", "col15", "col16", "col17"
]


df["tmax"] = pd.to_numeric(df["tmax"], errors="coerce")


krakow_obs = df[df["station_name"].str.contains("KRAK", na=False)]


hot_days = krakow_obs[(krakow_obs["month"] == 8) & (krakow_obs["tmax"] > 27)]


print(hot_days[["year", "month", "day", "tmax"]])


     year  month  day  tmax
652  2024      8    2  27.6
657  2024      8    7  27.8
658  2024      8    8  28.5
660  2024      8   10  28.2
661  2024      8   11  28.8
662  2024      8   12  28.0
663  2024      8   13  28.0
664  2024      8   14  29.2
665  2024      8   15  31.9
666  2024      8   16  32.3
667  2024      8   17  31.3
668  2024      8   18  29.8
669  2024      8   19  28.9
671  2024      8   21  28.0
673  2024      8   23  28.3
674  2024      8   24  33.0
675  2024      8   25  33.6
676  2024      8   26  27.4
677  2024      8   27  27.2
678  2024      8   28  30.5
679  2024      8   29  31.7
680  2024      8   30  32.4
681  2024      8   31  30.0


## Task 3: Select Landsat 8 Images Matching These Dates
- In Earth Engine, use `LANDSAT/LC08/C02/T1_L2` collection.
- Apply filters:
  - `.filterBounds(aoi)`
  - `.filterDate()` for the matching day
  - `.filterMetadata('CLOUD_COVER', 'less_than', 20)`
- Try to find **one image from 2013** and **one image from 2024** that match your hot days list.

In [61]:
import geemap
import ee 
start_2013 = ee.Date('2013-08-07')
end_2013 = ee.Date('2013-08-09')
start_2024 = ee.Date('2024-08-25')
end_2024 = ee.Date('2024-08-30')

image_2013 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterBounds(aoi).filterDate(start_2013, end_2013).filterMetadata('CLOUD_COVER', 'less_than', 20).first()
image_2024 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2').filterBounds(aoi).filterDate(start_2024, end_2024).filterMetadata('CLOUD_COVER', 'less_than', 20).first()
image_2013 = image_2013.clip(aoi)
image_2024 = image_2024.clip(aoi)

print(image_2013.getInfo())
print(image_2024.getInfo())

vis_params = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 5000,
    'max': 15000,
    'gamma': 1.3
}
center = aoi.centroid(10).coordinates().reverse().getInfo()

Map = geemap.Map(center=center, zoom=15)

Map.addLayer(image_2013, vis_params, 'Landsat 2013')
Map.addLayer(image_2024, vis_params, 'Landsat 2024')
Map.addLayer(aoi, {'color': 'red'}, 'AOI')

Map

{'type': 'Image', 'bands': [{'id': 'SR_B1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [54, 25], 'origin': [4027, 4668], 'crs': 'EPSG:32634', 'crs_transform': [30, 0, 303885, 0, -30, 5690715]}, {'id': 'SR_B2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [54, 25], 'origin': [4027, 4668], 'crs': 'EPSG:32634', 'crs_transform': [30, 0, 303885, 0, -30, 5690715]}, {'id': 'SR_B3', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [54, 25], 'origin': [4027, 4668], 'crs': 'EPSG:32634', 'crs_transform': [30, 0, 303885, 0, -30, 5690715]}, {'id': 'SR_B4', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [54, 25], 'origin': [4027, 4668], 'crs': 'EPSG:32634', 'crs_transform': [30, 0, 303885, 0, -30, 5690715]}, {'id': 'SR_B5', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensi

Map(center=[50.100236451862415, 19.95833250000626], controls=(WidgetControl(options=['position', 'transparent_…

## Task 4: Process Thermal Band (ST_B10)
- Convert Band 10 to Brightness Temperature (Kelvin):
  `TB = ST_B10 * 0.00341802 + 149.0`
- Create a visualization of each image using the same color scale.

In [ ]:

def calc_brightness_temp(image):
    bt = image.select('ST_B10').multiply(0.00341802).add(149.0).rename('BT')
    return image.addBands(bt)


image_2013_bt = calc_brightness_temp(image_2013)
image_2024_bt = calc_brightness_temp(image_2024)


bt_vis_params = {
    'bands': ['BT'],
    'min': 290,
    'max': 320,
    'palette': ['blue', 'cyan', 'green', 'yellow', 'red']
}


Map = geemap.Map(center=center, zoom=15)
Map.addLayer(image_2013_bt.select('BT'), bt_vis_params, 'Brightness Temp 2013')
Map.addLayer(image_2024_bt.select('BT'), bt_vis_params, 'Brightness Temp 2024')
Map.addLayer(aoi, {'color': 'red'}, 'AOI')
Map


Map(center=[50.100236451862415, 19.95833250000626], controls=(WidgetControl(options=['position', 'transparent_…

## Task 5: Compare and Interpret
- Compare the two maps.
- Optionally calculate difference: `TB_2024 - TB_2013`
- Discuss: did the surface temperature increase in the area?
- Is there evidence of an Urban Heat Island effect related to development?

In [ ]:

bt_2013 = image_2013_bt.select('BT')
bt_2024 = image_2024_bt.select('BT')
bt_diff = bt_2024.subtract(bt_2013).rename('BT_Diff')


bt_diff_vis = {
    'min': -5,
    'max': 5,
    'palette': ['blue', 'white', 'red']
}


Map = geemap.Map(center=center, zoom=15)
Map.addLayer(bt_diff, bt_diff_vis, 'Temperature Difference (2024 - 2013)')
Map.addLayer(aoi, {'color': 'black'}, 'AOI')
Map


Map(center=[50.100236451862415, 19.95833250000626], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
#Analiza różnic temperatur powierzchniowych (Brightness Temperature) pomiędzy 2013 a 2024 rokiem ujawnia istotny wzrost temperatury w północno-zachodniej części AOI. 
#Obszar ten, obecnie mocno zurbanizowany, wykazuje charakterystyczne cechy miejskiej wyspy ciepła (UHI). 
#Może to być efektem zagęszczenia zabudowy oraz ograniczenia terenów zielonych. Na pozostałym obszarze zmiany są minimalne lub niezauważalne, co sugeruje względną stabilność pokrycia terenu.